In [25]:
import pandas as pd
import numpy as np
import os # 20250406 George

In [26]:
# Get current working directory

current_dir = os.getcwd()  # 20250403 George added

# code_dir= "C:\\Users" +'\\' + username + '\\' + "Equitable Bank\\EQB-Concentra Fit for Use - Fit_for_Use Development - Fit_for_Use Development\\RWA\\code" # 20250403 George commented out Francesca/Joseph's code

code_dir = current_dir # 20250406 George added

snapshot_num = input('Enter the snapshot needed in the format YYYYMM, e.g. 202212: ') # 20250406 George added

if int(snapshot_num) == 202412:
    input_dir = code_dir + '\\..\\' + "Inputs" # 20250406 George added
    Output_dir = code_dir + '\\..\\' + "Outputs" #20250406 George added

if int(snapshot_num) == 202212:
    input_dir = code_dir + '\\..\\' + "Dec. 2022 RWA Inputs" #20250410 George added
    Output_dir = code_dir + '\\..\\' + "Replicated Dec 2022 Outputs" #20250410 George added

In [27]:
print(Output_dir) # 20250406 George added

c:\Users\georgez\OneDrive - Equitable Bank\MD_RWA_Impact_Analysis (Joseph)\SFR\code\..\Outputs


In [28]:
# read parquet file

eqb_rwa = pd.read_parquet(Output_dir + '\\eqb_rwa_original.parquet', engine='pyarrow') # 20250406 George added

In [29]:
# read parquet file

eqb_rwa_addon_zero = pd.read_parquet(Output_dir + '\\eqb_rwa_addon_zero.parquet', engine='pyarrow') # 20250406 George added

In [30]:
eqb_rwa.columns.values

array(['Loan_Number', 'Insured_class', 'EAD', 'Advance_Amount',
       'Years_to_maturity', 'corr_uninsured', 'RWA_standardized',
       'SL_Date', 'CalibratedPD', 'MRS_Bin_PD', 'Insured_Ind',
       'Alt_Prime_Indicator', 'RemainingPrincipal_Tot_Exp',
       'RemainingPrincipal_Excl_Partner', 'Corporate_Applicant_Ind',
       'Combo_Province_Metro_Override_WOE', 'Combo_LTV_Insured_Ind',
       'MRS_Bin_LGD', 'LR_Avg_LGD', 'Segment_Avg_LGD', 'Base_Line_LGD',
       'LGD_DT_Adjusted', 'Pre_final_LGD', 'Occupancy', 'LoanNumber',
       'AddOn', 'DLGD_floor', 'OSFI_LGD_floor', 'Final_LGD',
       'LGD_Before_Floors', 'Model_LGD_Modified', 'EAD_DLGD_Modified',
       'deductible_amount', 'insured_PMI_ratio', 'EAD_Post_CRM',
       'Maturity_adj', 'corr_insured', 'risk_weight_uninsured',
       'risk_weight_insured', 'RWA_new'], dtype=object)

In [31]:
#pd2022 = pd.read_csv(r'C:\Users\jasonz\OneDrive - Equitable Bank\Fit_for_Use Development\git_repos_MD\jasonz\input_data\MRS_Replcation_Check (Sent to Abhi 20240221).csv')
# pd_2024 = pd.read_parquet("C:\\Users\\francescav\\OneDrive - Equitable Bank\\EQB-Concentra Fit for Use - Fit_for_Use Development - Fit_for_Use Development\\RWA\code\\Inputs\\Rating_July2020_to_Dec2024_EQB_wAIRBScopeExcl_and_PostDefault_indicators (from Francesca 20250403).parquet")  #20250403 George commented out Francesca's code

# pd_2024 = pd.read_parquet(input_dir + "\\Rating_July2020_to_Dec2024_EQB_wAIRBScopeExcl_and_PostDefault_indicators (from Francesca 20250403).parquet")  # 20250414 George commented out Francesca's code



#pd_2022=pd2022.query("AIRB_Scope_Exclusion_Flag==0 and Post_Default_Ind==0 and SL_Date=='2022-12-31'")
#pd2024['Sub_Product'].unique()
#pd2024['LoanType'].unique()
#pd2024.columns.values
'''
Already added in filtered in the above dataset
pd2024['AIRB_Scope_Exclusion_Flag']=0 # 20250403 Fran added exclusion ind for 2024_12_31
pd2024.loc[pd2024.Sub_Product.isin(['GMAC', 'Street Capital']) ,'AIRB_Scope_Exclusion_Flag']=1.0 
pd2024.loc[pd2024.LoanType.isin([150]) ,'AIRB_Scope_Exclusion_Flag']=1.0 
pd_2024=pd2024.query("AIRB_Scope_Exclusion_Flag==0 and Post_Default_Ind==0 and SL_Date=='2024-12-31'")
'''

'\nAlready added in filtered in the above dataset\npd2024[\'AIRB_Scope_Exclusion_Flag\']=0 # 20250403 Fran added exclusion ind for 2024_12_31\npd2024.loc[pd2024.Sub_Product.isin([\'GMAC\', \'Street Capital\']) ,\'AIRB_Scope_Exclusion_Flag\']=1.0 \npd2024.loc[pd2024.LoanType.isin([150]) ,\'AIRB_Scope_Exclusion_Flag\']=1.0 \npd_2024=pd2024.query("AIRB_Scope_Exclusion_Flag==0 and Post_Default_Ind==0 and SL_Date==\'2024-12-31\'")\n'

In [32]:
#pd_2022.shape
# pd_2024.shape

In [33]:
#pd_2022.columns.values
# pd_2024.columns.values

In [34]:
#eqb_rwa= eqb_rwa.merge(pd_2022[['Loan_Number','Combo_Province_Metro_Override','Combo_Province_Metro_Override_WOE','Combo_LTV_Insured_Ind']],on='Loan_Number', how='left')
# eqb_rwa= eqb_rwa.merge(pd_2024[['Loan_Number','Combo_Province_Metro_Override','Combo_Province_Metro_Override_WOE','Combo_LTV_Insured_Ind']],on='Loan_Number', how='left')  # 20250414 eorge comment out Francesca's code because these columns are already there

In [35]:
# eqb_rwa_addon_zero= eqb_rwa_addon_zero.merge(pd_2024[['Loan_Number','Combo_Province_Metro_Override',  'Combo_Province_Metro_Override_WOE','Combo_LTV_Insured_Ind']],on='Loan_Number', how='left')  #20250414 George comment out because the columns already are there

In [36]:
eqb_rwa.Insured_Ind.value_counts()

Not Insured    28563
Insured        24443
Name: Insured_Ind, dtype: int64

In [37]:
summ=eqb_rwa.query("Insured_Ind=='Not Insured'").groupby(['Combo_LTV_Insured_Ind','Combo_Province_Metro_Override_WOE'],dropna=False).RWA_new.agg(['count','sum'])

In [38]:
summ_addon_zero=eqb_rwa_addon_zero.query("Insured_Ind=='Not Insured'").groupby(['Combo_LTV_Insured_Ind','Combo_Province_Metro_Override_WOE'],dropna=False).RWA_new.agg(['count','sum']).rename(columns={'count':'count_addon_zero','sum':'sum_addon_zero'})

In [39]:
add_on_impact = pd.concat([summ,summ_addon_zero],axis=1)

In [40]:
add_on_impact['impact']=add_on_impact['sum']/add_on_impact['sum_addon_zero']

In [41]:
add_on_impact.reset_index(inplace=True)

# Save Data

In [42]:
# Save Data
#add_on_impact.to_excel(r'C:\\Users\\jasonz\\OneDrive - Equitable Bank\\Fit_for_Use Development\\RWA\\addon_impact.xlsx')
# add_on_impact.to_excel(r'C:\\Users\\francescav\\OneDrive - Equitable Bank\\EQB-Concentra Fit for Use - Fit_for_Use Development - Fit_for_Use Development\\RWA\code\\Outputs\\\addon_impact.xlsx') # 20250406 George commented out Francesca's code

add_on_impact.to_excel(Output_dir + '\\addon_impact.xlsx') # 20250406 George added